# Lets save our data as a huggingface dataset, so it's quick to reuse



In [ ]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


In [ ]:
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
# from src.datasets.scores import choice2ids, scores2choice_probs

# Params

In [ ]:
# Params
BATCH_SIZE = 1  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True

from src.extraction.config import ExtractConfig

cfg = ExtractConfig(
    # model="HuggingFaceH4/starchat-beta",
    # model="TheBloke/CodeLlama-13B-Instruct-fp16", # too large!
    model="WizardLM/WizardCoder-3B-V1.0",
    # model="WizardLM/WizardCoder-1B-V1.0",
    # model="WizardLM/WizardCoder-Python-7B-V1.0", # too large!
    datasets = [
        "imdb", 
                ],
    max_examples=(400, 312),
)
cfg

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large coding ones might be best for lying.

In [ ]:
from src.models.load import verbose_change_param, AutoConfig, AutoTokenizer, AutoModelForCausalLM

def load_model(model_repo = "HuggingFaceH4/starchat-beta"):
    # see https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/starchat.py
    model_options = dict(
        device_map="auto",
        # load_in_8bit=True,
        # load_in_4bit=True,
        torch_dtype=torch.float16, # note because datasets pickles the model into numpy to get the unique datasets name, and because numpy doesn't support bfloat16, we need to use float16
        # use_safetensors=False,
    )

    config = AutoConfig.from_pretrained(model_repo, use_cache=False)
    verbose_change_param(config, 'use_cache', False)
    
    tokenizer = AutoTokenizer.from_pretrained(model_repo)
    verbose_change_param(tokenizer, 'pad_token_id', 0)
    verbose_change_param(tokenizer, 'padding_side', 'left')
    verbose_change_param(tokenizer, 'truncation_side', 'left')
    
    model = AutoModelForCausalLM.from_pretrained(model_repo, config=config, **model_options)

    return model, tokenizer

model, tokenizer = load_model(cfg.model)

# Scratch

In [ ]:
token_y = tokenizer(' True').input_ids
token_n = tokenizer(' Fakse').input_ids

# Load Dataset

In [ ]:

from itertools import chain, islice
from datasets import Dataset
import functools
# from datasets.arrow_dataset import Dataset
from src.prompts.prompt_loading import load_prompts

@functools.lru_cache()
def count_tokens(s):
    return len(tokenizer(s).input_ids)

def answer_len(answer_choices: list):
    a = count_tokens(answer_choices[0])
    b = count_tokens(answer_choices[1])
    return max(a, b)


def sample_n_true_y_false_prompts(prompts, num_truth=1, num_lie=1, seed=42):
    """sample some truth and some false"""
    df = pd.DataFrame(prompts)
    
    # restrict to template where the choices are a single token
    m = df.answer_choices.map(answer_len)<=2
    df = df[m]
    df = pd.concat([
        df.query("instructed_to_lie==True").sample(num_truth, random_state=seed),
        df.query("instructed_to_lie==False").sample(num_lie, random_state=seed)])
    return df.to_dict(orient="records")

        
# loop through all prompts in this dataset
ds_names = cfg.datasets
split_type = "train"

ds_name = ds_names[0]
prompt_ds = load_prompts(
    ds_name,
    num_shots=cfg.num_shots,
    split_type=split_type,
    template_path=cfg.template_path,
    seed=cfg.seed,
    prompt_format='llama'
)

# for each example, sample true and false
N = cfg.max_examples[split_type!="train"]
g = map(lambda r: sample_n_true_y_false_prompts(r[1], seed=r[0]+cfg.seed), enumerate(prompt_ds))

# and combine them into one big list
g = chain.from_iterable(g) 
prompt_ds2 = list(tqdm(islice(g, N), total=N))

# convert to hugginface dataset
dataset = Dataset.from_list(prompt_ds2)
dataset

In [ ]:
b = next(iter(prompt_ds))
b
sample_n_true_y_false_prompts(b)

## Format prompts

The prompt is the thing we most often have to change and debug. So we do it explicitly here.

We do it as transforms on a huggingface dataset.

In this case we use multishot examples from train, and use the test set to generated the hidden states dataset. We will test generalisation on a whole new dataset.


In [ ]:
from src.datasets.scores import scores2choice_probs
from src.datasets.scores import choice2id, choice2ids

def row_choice_ids(r):
    return choice2ids([[c] for c in r['answer_choices']], tokenizer)

In [ ]:
ds = (
    dataset
    .map(
        lambda ex: tokenizer(
            ex["question"], padding="max_length", max_length=600, truncation=True, add_special_tokens=True,
            # return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True,
    )
    .map(
        lambda r: {"prompt_truncated": tokenizer.batch_decode(r["input_ids"])},
        batched=True,
    )
    .map(lambda r: {'choice_ids': row_choice_ids(r)})
)
ds

# Scratch

In [ ]:
# ds[0].keys()

torch_cols = ['input_ids', 'attention_mask', 'choice_ids']

ds_o = ds.remove_columns(torch_cols)
ds.set_format('torch', torch_cols)
row = ds[0]
row_0 = ds_o[0]
row.keys()
# prompt =row['question']
# prompt
# row.keys()

In [ ]:
row_0.keys()

In [ ]:
model

In [ ]:
input_ids, attention_mask, choice_ids = row['input_ids'].to(model.device)[None, :], row['attention_mask'].to(model.device)[None, :], row['choice_ids'].to(model.device)[None, :]
choice_ids

note bigcode vs normal llamba. one has self attention one has cross
- [llama2](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py)
- [gpt_bigcode](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_bigcode/modeling_gpt_bigcode.py)


and

- [honest_llama](https://github.com/likenneth/honest_llama/blob/e010f82bfbeaa4326cef8493b0dd5b8b14c6da67/utils.py#L159)


and

- [tracedict](https://github.com/davidbau/baukit/blob/main/baukit/nethook.py)

In [ ]:
def get_gradients(model, scores, token_y, token_n):
    model.zero_grad()
    assert token_y.shape[1]<2, 'FIXME just use the first token for now'
    score_y = torch.index_select(scores, 1, token_y[:, 0])
    score_n = torch.index_select(scores, 1, token_n[:, 0])
    pred = score_y - score_n
    loss = F.mse_loss(pred, -pred)
    loss.backward()


In [ ]:
import gc
output = scores = None
model.eval()
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from baukit import Trace, TraceDict
HEADS = [f"transformer.h.{i}.attn.c_proj" for i in range(model.config.num_hidden_layers)]
MLPS = [f"transformer.h.{i}.mlp" for i in range(model.config.num_hidden_layers)]
model.train()
with torch.autocast('cuda' dtype=torch.bfloat16):
    with TraceDict(model, HEADS+MLPS, retain_grad=True) as ret:
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True)
        scores = outputs.logits[:, -1, :]
        
        token1_n = choice_ids[:, 0] # [batch, tokens]
        token1_y = choice_ids[:, 1]
        get_gradients(model, scores, token1_y, token1_n)

In [ ]:
len(HEADS), len(MLPS)

In [ ]:
# head_wise_hidden_states = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
# torch.stack(head_wise_hidden_states, dim=0)[:, -1].squeeze().numpy().shape
def stack_trace_returns(ret: TraceDict, HEADS: List[str]) -> torch.Tensor:
    hs = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
    return torch.stack(hs, dim=0).squeeze().numpy()[:, -1]

hidden_states = torch.stack(outputs.hidden_states, dim=0).squeeze()
hidden_states = hidden_states.detach().cpu().numpy()[:, -1]

head_wise_hidden_states = stack_trace_returns(ret, HEADS)
mlp_wise_hidden_states = stack_trace_returns(ret, MLPS)
hidden_states.shape, head_wise_hidden_states.shape, mlp_wise_hidden_states.shape

In [ ]:
a = ret['transformer.h.0.attn.c_proj']
a.output.grad.shape, a.output.shape
# dir(a)

In [ ]:
a = ret['transformer.h.0.mlp']
a.output.grad.shape, a.output.shape
